### Dependencies & working functions

In [ ]:
%%shell
# update "apt" which is the local programs on this virtual machine that is Debian Based
sudo apt -y update -q

# install curl, wget, and unzip - these utilities will help us retrieve files online and unzip them as needed.
sudo apt install -y wget curl unzip -q

# Colab has updated from ubuntu 18.04 LTS to 20.04 LTS (LTS stands for "Long-Term Support") Think of google doing this as an upgrade from one Windows version to another.
# To summarize whats below: We are getting the correct chrome driver for the "stable_current" verson of chrome used by colab.
# {
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
# }

pip install selenium -q

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,299 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,360 kB]
Hit:13 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:14 http://secu

In [ ]:
from selenium import webdriver
import pandas as pd
import numpy as np
import os
import time
import random
from bs4 import BeautifulSoup

# Disable all warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def print_loading_bar(current, total, time, bar_length=40):
    progress = current / total
    completed_length = int(bar_length * progress)
    remaining_length = bar_length - completed_length

    bar = '█' * completed_length + '-' * remaining_length

    print(f"\r[{bar}] Processing {current} of {total} COMPLETE {progress*100:.2f}% Time Lapsed {time:.2f} Seconds", end ="")

def load_join(select, loaded):
    selected_df = pd.read_csv("https://www.dropbox.com/s/" + select + "?dl=1", index_col="review_index")
    if loaded is not None:
        selected_df = pd.merge(loaded, selected_df, on='review_index')
    return selected_df

In [ ]:
def search_business_industry(business_name):
    url = f"https://www.glassdoor.com/Search/results.htm?keyword={business_name}"

    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    custom_user_agent = "Mozilla/5.0 (Linux; Android 10; K) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Mobile Safari/537.36"
    options.add_argument(f"--user-agent={custom_user_agent}")

    driver = webdriver.Chrome(options=options)

    driver.get(url)
    sleep_time = random.randint(15, 25)
    time.sleep(sleep_time)

    html = driver.page_source
    driver.quit()

    return html

### Load Data

In [ ]:
load_metadata = True
load_ratings = False
load_opinions = False
load_descriptions = False
load_world_happiness_report = True
word_counted_preprocessing_bypass = False

index_df = None
if load_metadata == True:
    load_index = "goqyfyeu2qvpsmo/metadata_df.csv"
    index_df = load_join(load_index, index_df)
if load_ratings == True:
    load_index = "9j2j86xwqrmrljx/ratings_df.csv"
    index_df = load_join(load_index, index_df)
if load_opinions == True:
    load_index = "q8v9f6rbb4z12df/opinion_df.csv"
    index_df = load_join(load_index, index_df)
if load_descriptions == True:
    load_index = "hnp8sebleh6dzgt/descriptions_df.csv"
    index_df = load_join(load_index, index_df)

if load_world_happiness_report == True:
    WHR_df = pd.read_csv("https://www.dropbox.com/s/jyr4e7fleevrb1s/WHR2023.csv?dl=1")

if word_counted_preprocessing_bypass == True:
    joint_df = pd.read_csv("https://www.dropbox.com/s/e64gm79gxu1tgip/joint_df.csv?dl=1")

In [ ]:
working_df = index_df.copy()
working_WHR_df = WHR_df.copy()

### Processing and Cleaning Data

In [ ]:
working_firm_column = working_df['firm'].copy()
unique_firms = working_firm_column.unique().tolist()

unique_firms_length = len(unique_firms)
chunk_size = unique_firms_length // 4

# Split the list into four smaller lists
unique_firms_part1 = unique_firms[:chunk_size]
unique_firms_part2 = unique_firms[chunk_size:2*chunk_size]
unique_firms_part3 = unique_firms[2*chunk_size:3*chunk_size]
unique_firms_part4 = unique_firms[3*chunk_size:]

# split the list into smaller chunks to avoid long scraping times.

# Print the resulting smaller lists
print("unique_firms_part1:")
print(unique_firms_part1)
print()

print("unique_firms_part2:")
print(unique_firms_part2)
print()

print("unique_firms_part3:")
print(unique_firms_part3)
print()

print("unique_firms_part4:")
print(unique_firms_part4)
print()

<class 'list'>


In [ ]:
start_time = time.time()
index = 0
part_one_html_data = []

for firm in unique_firms_part1:
    index = index + 1
    current_time = time.time()
    execution_time = current_time - start_time
    print_loading_bar(index, len(unique_firms), execution_time)
    firm_html = search_business_industry(firm)

[████████████████████████████████████████] Processing 428 of 428 COMPLETE 100.00% Time Lapsed 0.09 Seconds

In [ ]:
with open("test.txt", "w") as file:
    file.write(html)